In [1]:
from openai import OpenAI

client = OpenAI()

assistant = client.beta.assistants.create(
    name="Book Assistant v2",
    instructions="You help users with their questions on the files they upload.",
    tools=[{"type": "file_search"}],
    model="gpt-4o",
)

# assistant_id = "asst_R0iQNbINw8zNsXGeppTnBZYD"

In [2]:
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I want you to answer my questions with the files I uplaoded. When you answer my question, you MUST include the source citations with the full source text at the bottom so I can easily find the sources.",
)

thread

Thread(id='thread_nHtnljPqee7IWeELvZ4t3tFj', created_at=1720486743, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [13]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

print(run)

Run(id='run_fQ7N9Oeho7oT6YBjDo9yJ2yp', assistant_id='asst_oJaux3brfP46PF1oRVNVXTYn', cancelled_at=None, completed_at=None, created_at=1720487379, expires_at=1720487979, failed_at=None, incomplete_details=None, instructions='You help users with their questions on the files they upload.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_nHtnljPqee7IWeELvZ4t3tFj', tool_choice='auto', tools=[FileSearchTool(type='file_search', file_search=None)], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})


In [ ]:
with open("./files/Orwell_1984.pdf", "rb") as file_object:
    file = client.files.create(
        file=file_object,
        purpose="assistants",
    )

file

FileObject(id='file-1VdmHgsBB8e1rNRARlHoi1JL', bytes=1644307, created_at=1720486747, filename='Orwell_1984.pdf', object='file', purpose='assistants', status='processed', status_details=None)

In [4]:
client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Please refer to the uploaded file for my question. ",
    attachments=[
        {
            "file_id": file.id,
            "tools": [
                {
                    "type": "file_search",
                }
            ],
        }
    ],
)

Message(id='msg_H5KAwcidptQSYeVKQOkeSkhC', assistant_id=None, attachments=[Attachment(file_id='file-1VdmHgsBB8e1rNRARlHoi1JL', tools=[AttachmentToolAssistantToolsFileSearchTypeOnly(type='file_search')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Please refer to the uploaded file for my question. '), type='text')], created_at=1720486750, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_nHtnljPqee7IWeELvZ4t3tFj')

In [5]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")


def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=content
    )

In [6]:
get_messages(thread.id)

user: I want you to answer my questions with the files I uplaoded.
user: Please refer to the uploaded file for my question. 


In [14]:
get_run(run.id, thread.id).status

'completed'

In [12]:
get_messages(thread.id)

user: I want you to answer my questions with the files I uplaoded.
user: Please refer to the uploaded file for my question. 
assistant: Based on the provided text from "1984" by George Orwell, I need you to clarify which specific aspect or topic from the book you're interested in. Are you looking for details on a particular character, concept (like doublethink or Newspeak), plot point, or thematic element? Please provide more specific information so that I can assist you effectively.
user: I want to know where Winston lives and how his residence looks like.


In [11]:
send_message(
    thread.id, "I want to know where Winston lives and how his residence looks like."
)

Message(id='msg_WNf1GGxbU8X5sdXKsyjyMM1g', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='I want to know where Winston lives and how his residence looks like.'), type='text')], created_at=1720487334, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_nHtnljPqee7IWeELvZ4t3tFj')

In [15]:
get_messages(thread.id)

user: I want you to answer my questions with the files I uplaoded.
user: Please refer to the uploaded file for my question. 
assistant: Based on the provided text from "1984" by George Orwell, I need you to clarify which specific aspect or topic from the book you're interested in. Are you looking for details on a particular character, concept (like doublethink or Newspeak), plot point, or thematic element? Please provide more specific information so that I can assist you effectively.
user: I want to know where Winston lives and how his residence looks like.
assistant: Winston Smith lives in a flat in Victory Mansions in London. The building and his flat are described in rather bleak terms. The hallways of Victory Mansions smell of boiled cabbage and old rag mats, and the lift often does not work due to frequent power cuts as part of the Party's economy drive. His flat is fitted with a telescreen, which cannot be turned off and constantly monitors him, making privacy impossible【9:1†sour